In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

POPDENS_PATH = '/ihme/covid-19/deaths/mobility_inputs/2020_04_14/pop_density.csv'
COVID_PATH = '/ihme/covid-19/deaths-outputs/best/model_results.csv'
COVID_POP_PATH = '/ihme/covid-19/deaths-outputs/best/model_data.csv'


In [ ]:
popdens_df = pd.read_csv(POPDENS_PATH)

In [ ]:
pop_df = pd.read_csv(COVID_POP_PATH)
pop_df = pop_df[['location_id', 'population']].drop_duplicates()
covid_df = pd.read_csv(COVID_PATH)
covid_df['date'] = pd.to_datetime(covid_df['date'])
covid_df['deaths'] = covid_df[[f'draw_{d}' for d in range(1000)]].mean(axis=1)
covid_df = covid_df.drop([f'draw_{d}' for d in range(1000)], axis=1)

covid_df = covid_df.sort_values(['location_id', 'date']).reset_index(drop=True)
first_date = covid_df['date'] == covid_df.groupby('location_id')['date'].transform(min)
covid_df['Daily deaths'] = np.diff(covid_df['deaths'], prepend=np.nan)
covid_df.loc[first_date, 'Daily deaths'] = covid_df['deaths']
covid_df = covid_df.groupby('location_id', as_index=False)['Daily deaths'].max()
covid_df = covid_df.merge(pop_df)
covid_df['peak_rate'] = covid_df['Daily deaths'] / covid_df['population']


In [ ]:
df = popdens_df[['location_id', 'location_name', 'pop_density', 'pop_proportion']].merge(covid_df)

In [ ]:
pds = df['pop_density'].unique().tolist()
fig, ax = plt.subplots(3, 3, figsize=(22, 16))
for i, pd in enumerate(pds):
    col = int(i/3)
    row = i % 3
    plot_df = df.loc[df['pop_density'] == pd].reset_index(drop=True)
    ax[col, row].scatter(plot_df['pop_proportion'], plot_df['peak_rate'])
    ax[col, row].set_title(pd)
    

In [ ]:
df.loc[df['location_name'] == 'DC']